# Transformer 실습

## 목표

---

- [ ]  Last word prediction dataset 준비
    - 기존의 IMDB dataset을 그대로 활용하고, `collate_fn`을 다음과 같이 수정
        
        ```python
        from torch.nn.utils.rnn import pad_sequence
        
        def collate_fn(batch):
          max_len = 400
          texts, labels = [], []
          for row in batch:
            labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-3])
            texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-3]))
        
          texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
          labels = torch.LongTensor(labels)
        
          return texts, labels
        ```
        
- [ ]  Loss function 및 classifier output 변경
    - 마지막 token id를 예측하는 것이기 때문에 binary classification이 아닌 일반적인 classification 문제로 바뀜. MNIST 과제에서 했던 것 처럼 loss와 `TextClassifier`의 출력 차원을 잘 조정하여 task를 풀 수 있도록 수정
- [ ]  학습 결과 report
    - 기존 Transformer 실습에서 사용한 모델로 last word prediction을 학습하고 학습 경과를 report


In [1]:
!pip install datasets sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" 

In [23]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from torch.nn.utils.rnn import pad_sequence



# ds = load_dataset("stanfordnlp/imdb")
train_ds = load_dataset("stanfordnlp/imdb", split="train")
test_ds = load_dataset("stanfordnlp/imdb", split="test")

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')



def collate_fn(batch):
    max_len = 400
    texts, labels = [], []

    for row in batch:
        input_ids = tokenizer(row['text'], truncation=True, max_length=max_len).input_ids
        if len(input_ids) < 4:
            continue  # 최소 길이 확보
        labels.append(input_ids[-3])  # 마지막 단어 (예측 대상)
        texts.append(torch.LongTensor(input_ids[:-3]))  # 입력에서 제외

    texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = torch.LongTensor(labels)

    return texts, labels



train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [3]:
train_ds['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [4]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
output.tokens

['[CLS]',
 'hello',
 ',',
 'y',
 "'",
 'all',
 '!',
 'how',
 'are',
 'you',
 '[UNK]',
 '?',
 '[SEP]']

In [5]:
from torch import nn
from tokenizers import Tokenizer
import torch

tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
x = torch.LongTensor(tokenizer.encode("Hello, y'all! How are you 😁 ?").ids)[None]
x


tensor([[ 101, 7592, 1010, 1061, 1005, 2035,  999, 2129, 2024, 2017,  100, 1029,
          102]])

In [6]:
embedding = nn.Embedding(tokenizer.get_vocab_size(), 256)
embedding(x[None]).shape

torch.Size([1, 1, 13, 256])

In [7]:
print(x[0])

tensor([ 101, 7592, 1010, 1061, 1005, 2035,  999, 2129, 2024, 2017,  100, 1029,
         102])


In [9]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)
    result = self.dense(result)

    return result

In [10]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

In [11]:
import numpy as np
from torch.nn.utils.rnn import pad_sequence




def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)




torch.Size([1, 400, 256])


In [33]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)

    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])

    # self.classification = nn.Linear(d_model, 1)
    self.classification = nn.Linear(d_model, vocab_size) # vocab_size로 변경


  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


# model = TextClassifier(len(tokenizer), 32, 2, 32)
model = TextClassifier(
    vocab_size=tokenizer.vocab_size,
    d_model=128,
    n_layers=4,
    dff=512
)



## 학습


In [34]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [35]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
    cnt = 0
    acc = 0

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        preds = model(inputs)
        preds = torch.argmax(preds, dim=-1)  # 다중 클래스 분류 기준

        cnt += labels.shape[0]
        acc += (labels == preds).sum().item()

    return acc / cnt


In [36]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda') #.float()

    # preds = model(inputs)[..., 0]
    preds = model(inputs)  # shape: (B, vocab_size)
    loss = loss_fn(preds, labels)  # labels: (B,)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 2968.5435452461243
=========> Train acc: 0.035 | Test acc: 0.033
Epoch   1 | Train Loss: 2682.4187622070312
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   2 | Train Loss: 2660.7557015419006
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   3 | Train Loss: 2654.480152606964
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   4 | Train Loss: 2650.774440765381
=========> Train acc: 0.039 | Test acc: 0.039


KeyboardInterrupt: 

## 학습효율 문제

* 왜 필터링이 필요할까?
- Transformer는 강력하긴 하지만 학습 효율이 다음의 요소에 매우 민감:

1. 라벨 클래스 수가 너무 많을 때 → 학습 signal이 희석됨

2. 라벨 분포가 한쪽으로 쏠려있을 때 → 모델이 특정 토큰만 예측하려 함

3. 불필요하거나 학습에 도움 안 되는 예시가 많을 때 → 학습 리소스 낭비

정확히 짚으셨어요 👏  
지금 하는 **“마지막 단어 예측”** task는 다중 클래스 분류 중에서도 **가장 난이도가 높은 종류**예요. 단어 수(클래스 수)가 수만 개고, 문장 끝 단어는 예측이 특히 어렵습니다.

그래서 **"데이터 필터링"**을 통해 학습 효율을 높이는 건 매우 좋은 접근이고, 여러 방법이 가능합니다. 아래에서 **전략별 고민 포인트 + 필터링 방식 + 장단점**을 함께 정리할게요.

---

## 🧰 **고려할 수 있는 데이터 필터링 전략**

---

### 1. **Rare token 제거**  

**아이디어:**  
전체 dataset을 스캔해서 마지막 단어(예측 대상) 중 너무 드물게 나오는 토큰은 제거하자.

**적용 이유:**  
드물게 나오는 단어는 모델이 학습할 기회도 적고, 확률적으로 맞출 가능성도 거의 없음. 이걸 계속 학습하게 두면 **loss는 커지고 gradient는 noise만 생김**.

**장점:**
- 성능 안정화
- 학습 효율 대폭 향상
- loss 폭 감소

**단점:**
- 드문 but 중요한 토큰 예측 능력은 손실될 수 있음
- 라벨 bias 생길 수 있음 (자주 나오는 것만 맞추는 습관 생김)

---

### 2. **짧은 문장 제거**

**아이디어:**  
너무 짧은 문장 (예: 5단어 이하)은 context가 부족해서 마지막 단어를 추측할 정보가 없음.

**적용 예시:**
```python
if len(input_ids) < 10: continue
```

**장점:**
- 문맥 부족한 input 제거 → 모델이 무의미한 학습 안 함
- positional encoding, attention 효과 증가

**단점:**
- 데이터 수가 줄어듦
- 문장 길이 다양성 손실

---

### 3. **Stopword / 구두점 제거**

**아이디어:**  
마지막 단어가 '.', ',', 'the', 'a', 'I' 같은 자주 등장하지만 예측 의미가 없는 토큰이면 제거

**적용 예시:**
```python
if tokenizer.decode([label]).lower() in stop_words:
    continue
```

**장점:**
- 실제로 의미 있는 예측만 남음
- 학습된 모델이 문장 생성/예측에서 훨씬 자연스러움

**단점:**
- stopword 정의가 어렵고 language-specific

---


## ✅ **"빈도수 기반 rare token 제거"**

- 코드 구현이 간단하고 빠르게 적용 가능
- vocab 수를 줄여서 **softmax 연산 부담 완화**
- 정확도와 loss 개선을 **단기간에 체감할 수 있음**

---

## 🔁 결론 & 다음 단계

| 목적 | 방법 | 실행 난이도 | 추천 순위 |
|------|------|--------------|------------|
| 라벨 다양성 줄이기 | 빈도수 기반 rare token 제거 | ⭐️ 쉽다 | ✅ 1순위 |
| 의미 없는 문장 제거 | 짧은 문장 제거 | ⭐️ 쉽다 | ✅ 1~2순위 |
| noise 제거 | stopword 필터링 | ⚠️ 중간 | ⏳ 실험 필요 |
| 언어모델 pretrain처럼 | 중간 위치 단어 예측 | ⚠️ 어렵다 | ⏳ 구조 변경 필요 |

---


## 학습 데이터 필터링 : **Rare token 제거**  
collate_fn에서 너무 짧거나 rare한 label은 제외:

In [40]:
from collections import Counter

# 사전적으로 label 빈도수 구해놓고
label_counts = Counter()

for row in train_ds:
    input_ids = tokenizer(row['text'], truncation=True, max_length=400).input_ids
    if len(input_ids) >= 4:
        label_counts[input_ids[-3]] += 1

common_labels = set([tok for tok, cnt in label_counts.items() if cnt > 10])  # 최소 10번 이상 등장한 label만

def collate_fn(batch):
    max_len = 400
    texts, labels = [], []

    for row in batch:
        input_ids = tokenizer(row['text'], truncation=True, max_length=max_len).input_ids
        if len(input_ids) < 4:
            continue
        label = input_ids[-3]
        if label not in common_labels:
            continue  # rare token이면 제외
        labels.append(label)
        texts.append(torch.LongTensor(input_ids[:-3]))

    if len(texts) == 0:
        return None  # skip batch

    texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = torch.LongTensor(labels)
    return texts, labels


In [ ]:
model = TextClassifier(
    vocab_size=tokenizer.vocab_size,
    d_model=64,
    n_layers=2,
    dff=256
)

model = model.to('cuda')

n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    if data is None or data[0].numel() == 0:
        continue  # ← 빈 배치 skip

    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda') #.float()

    # preds = model(inputs)[..., 0]
    preds = model(inputs)  # shape: (B, vocab_size)
    loss = loss_fn(preds, labels)  # labels: (B,)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

In [44]:
print(f"# of training examples: {len(train_loader.dataset)}")
print("# Train examples:", sum(1 for _ in train_loader))


# of training examples: 25000
# Train examples: 391
